In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/epic/pytorch/default/1/deberta_fold_0.pth
/kaggle/input/epic/pytorch/default/1/deberta_fold_1.pth
/kaggle/input/epic/pytorch/default/1/deberta_fold_3.pth
/kaggle/input/epic/pytorch/default/1/roberta_fold_1.pth
/kaggle/input/epic/pytorch/default/1/roberta_fold_4.pth
/kaggle/input/epic/pytorch/default/1/roberta_fold_0.pth
/kaggle/input/epic/pytorch/default/1/roberta_fold_3.pth
/kaggle/input/epic/pytorch/default/1/deberta_fold_2.pth
/kaggle/input/epic/pytorch/default/1/roberta_fold_2.pth
/kaggle/input/epic/pytorch/default/1/deberta_fold_4.pth
/kaggle/input/neural3/test_complaints.csv
/kaggle/input/neural3/train_complaints.csv


In [ ]:

import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import re


TRAIN_CSV_PATH = "/kaggle/input/neural3/train_complaints.csv" 
TEST_CSV_PATH = "/kaggle/input/neural3/test_complaints.csv"
MODEL_FOLDER = "/kaggle/input/epic/pytorch/default/1"   


class Config:
    max_len = 512
    batch_size = 16   
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    TTA_LOOPS = 2      
    TEMP = 1.1        

def augment_text(text, loop_idx):
    """Creates variations of the text for TTA"""
    if loop_idx == 0: 
        return text                     
    elif loop_idx == 1: 
        return text.lower()             
    elif loop_idx == 2:
        return re.sub(r'[^\w\s]', '', text) 
    return text


class TTADataset(Dataset):
    def __init__(self, df, tokenizer, loop_idx=0):
        self.text = df['complaint_text'].astype(str).values
        self.tokenizer = tokenizer
        self.loop_idx = loop_idx

    def __len__(self): return len(self.text)

    def __getitem__(self, index):
        
        text = augment_text(self.text[index], self.loop_idx)
        
        inputs = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=Config.max_len,
            padding='max_length', 
            truncation=True,
            return_attention_mask=True
        )
        return {
            'input_ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(inputs['attention_mask'], dtype=torch.long)
        }


class UniversalModel(nn.Module):
    def __init__(self, model_name, num_primary, num_secondary):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(0.3)
        self.primary_head = nn.Linear(768, num_primary)
        self.secondary_head = nn.Linear(768, num_secondary)
        self.severity_head = nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids, attention_mask=attention_mask)
        pooled_output = self.drop(outputs.last_hidden_state[:, 0, :])
        return {'primary': self.primary_head(pooled_output),
                'secondary': self.secondary_head(pooled_output),
                'severity': self.severity_head(pooled_output)}




print("⏳ Loading Data...")
train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)

le_p = LabelEncoder().fit(train_df['primary_category'])
le_s = LabelEncoder().fit(train_df['secondary_category'])
num_p = len(le_p.classes_)
num_s = len(le_s.classes_)


final_p_logits = np.zeros((len(test_df), num_p))
final_s_logits = np.zeros((len(test_df), num_s))
final_sev_preds = np.zeros(len(test_df))

model_files = sorted(glob.glob(os.path.join(MODEL_FOLDER, "*.pth")))
print(f"🔍 Found {len(model_files)} models.")

for model_path in model_files:
    
    if "deberta" in model_path.lower():
        model_name = "microsoft/deberta-v3-base"
    else:
        model_name = "roberta-base"
        
    print(f"\n🤖 Processing {os.path.basename(model_path)} ({model_name})...")
    
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = UniversalModel(model_name, num_p, num_s)
    model.load_state_dict(torch.load(model_path, map_location=Config.device))
    model.to(Config.device)
    model.eval()
    
    
    for tta_loop in range(Config.TTA_LOOPS):
        print(f"   ↳ TTA Pass {tta_loop+1}/{Config.TTA_LOOPS}...")
        
        dataset = TTADataset(test_df, tokenizer, loop_idx=tta_loop)
        loader = DataLoader(dataset, batch_size=Config.batch_size, shuffle=False, num_workers=2)
        
        preds_p, preds_s, preds_sev = [], [], []
        
        with torch.no_grad():
            for batch in tqdm(loader, leave=False):
                input_ids = batch['input_ids'].to(Config.device)
                mask = batch['attention_mask'].to(Config.device)
                
                outputs = model(input_ids, mask)
                
               
                p_logits = outputs['primary'] / Config.TEMP
                s_logits = outputs['secondary'] / Config.TEMP
                
                
                preds_p.append(torch.softmax(p_logits, dim=1).cpu().numpy())
                preds_s.append(torch.softmax(s_logits, dim=1).cpu().numpy())
                preds_sev.append(outputs['severity'].squeeze().cpu().numpy())
        
        
        final_p_logits += np.concatenate(preds_p)
        final_s_logits += np.concatenate(preds_s)
        final_sev_preds += np.concatenate(preds_sev)

    del model, tokenizer
    torch.cuda.empty_cache()



print("\n✅ Inference Complete. Generating CSV...")


final_p_ids = np.argmax(final_p_logits, axis=1)
final_s_ids = np.argmax(final_s_logits, axis=1)


divisor = len(model_files) * Config.TTA_LOOPS
final_sev_preds /= divisor
final_sev_rounded = np.clip(np.round(final_sev_preds), 1.0, 5.0).astype(int)

submission = pd.DataFrame({
    'complaint_id': test_df['complaint_id'],
    'primary_category': le_p.inverse_transform(final_p_ids),
    'secondary_category': le_s.inverse_transform(final_s_ids),
    'severity': final_sev_rounded
})

submission.to_csv("submission_tta_boosted.csv", index=False)
print("🎉 Success! Saved 'submission_tta_boosted.csv'")
print(submission.head())

⏳ Loading Data...
🔍 Found 10 models.

🤖 Processing deberta_fold_0.pth (microsoft/deberta-v3-base)...


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
2026-02-08 11:05:29.584557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770548729.755319      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770548729.804515      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register fa

...

   ↳ TTA Pass 1/2...


  0%|          | 0/32 [00:00<?, ?it/s]

   ↳ TTA Pass 2/2...


  0%|          | 0/32 [00:00<?, ?it/s]


✅ Inference Complete. Generating CSV...
🎉 Success! Saved 'submission_tta_boosted.csv'
   complaint_id                                   primary_category  \
0       7799230  Credit reporting or other personal consumer re...   
1      15754196                                    Debt collection   
2      10989146  Credit reporting or other personal consumer re...   
3       3617850  Credit reporting, credit repair services, or o...   
4       5253879  Credit reporting or other personal consumer re...   

                                  secondary_category  severity  
0                        Improper use of your report         1  
1                    Written notification about debt         1  
2  Problem with a company's investigation into an...         1  
3  Problem with a credit reporting company's inve...         1  
4                        Improper use of your report         3  
